# Figures Graphical Abstract

## Setup and Helper Functions

In [ ]:
import json
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns
from biopsykit.utils.dataframe_handling import multi_xs
from fau_colors import cmaps, register_fausans_font

from pepbench.datasets import EmpkinsDataset, GuardianDataset
from pepbench.io import load_challenge_results_from_folder
from pepbench.plotting import (
    plot_signals_from_challenge_results,
    plot_signals_with_reference_labels,
)
from pepbench.plotting.algorithms import (
    plot_b_point_extraction_forouzanfar2018,
)
from pepbench.plotting.results import (
    boxplot_algorithm_performance,
    residual_plot_pep,
    residual_plot_pep_heart_rate,
)

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
register_fausans_font()
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", font="sans-serif", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "FAUSans Office"

palette

In [ ]:
deploy_type = "local"
config_dict = json.load(Path("../../config.json").open(encoding="utf-8"))

guardian_base_path = Path(config_dict[deploy_type]["guardian_path"])
empkins_base_path = Path(config_dict[deploy_type]["empkins_path"])

In [ ]:
root_path = Path("../../../pep_algorithm_benchmarking")

In [ ]:
results_path = root_path.joinpath("results")

In [ ]:
paper_path = json.load(root_path.joinpath("paper_path.json").open(encoding="utf-8"))["paper_path"]
paper_path = Path(paper_path)

img_path_paper = paper_path.joinpath("img/graphical_abstract")
img_path_paper.mkdir(parents=True, exist_ok=True)

In [ ]:
folder_path_empkins = results_path.joinpath("empkins_dataset_both_algorithms")
folder_path_empkins

In [ ]:
folder_path_guardian = results_path.joinpath("guardian_dataset_both_algorithms")
folder_path_guardian

In [ ]:
dataset_guardian = GuardianDataset(guardian_base_path, use_cache=True, only_labeled=True)
dataset_guardian

In [ ]:
dataset_empkins = EmpkinsDataset(empkins_base_path, use_cache=True, only_labeled=True)
dataset_empkins

In [ ]:
participant_empkins = "VP_001"
condition_empkins = "tsst"
phase_empkins = "Pause_1"

In [ ]:
datapoint_empkins = dataset_empkins.get_subset(
    participant=participant_empkins, condition=condition_empkins, phase=phase_empkins
)

In [ ]:
results_agg_mean_std_empkins, results_agg_total_empkins, results_single_empkins, results_per_sample_empkins = (
    load_challenge_results_from_folder(
        folder_path_empkins,
        index_cols_single=["participant", "condition", "phase"],
        index_cols_per_sample=["participant", "condition", "phase"],
    )
)

In [ ]:
results_per_sample_empkins

In [ ]:
results_agg_mean_std_guardian, results_agg_total_guardian, results_single_guardian, results_per_sample_guardian = (
    load_challenge_results_from_folder(
        folder_path_guardian,
        index_cols_single=["participant", "phase"],
        index_cols_per_sample=["participant", "phase"],
    )
)

In [ ]:
pipeline_result_empkins = results_per_sample_empkins.loc[("martinez2004", "drost2022", "none")]
pipeline_result_empkins = pipeline_result_empkins.xs(
    (participant_empkins, condition_empkins, phase_empkins), level=["participant", "condition", "phase"]
)

## Example Signal with Reference Labels

In [ ]:
fig, axs = plot_signals_with_reference_labels(
    datapoint_empkins,
    collapse=False,
    normalize_time=True,
    heartbeat_subset=(5, 6),
    legend_outside=True,
    plot_artefacts=True,
    legend_orientation="horizontal",
    figsize=(5, 4),
)
axs[0].set_ylabel(None)
axs[1].set_ylabel(None)

fig.legends[0].remove()
fig.tight_layout()
fig.savefig(img_path_paper.joinpath("img_signals.pdf"), facecolor=(1, 1, 1, 0))

## Example Signal with Reference Labels and Algorithm Output

In [ ]:
fig, axs = plot_signals_from_challenge_results(
    datapoint_empkins,
    pipeline_result_empkins,
    normalize_time=True,
    heartbeat_subset=(5, 6),
    collapse=False,
    add_pep=True,
    legend_outside=True,
    legend_orientation="horizontal",
    figsize=(5, 4),
)
axs[0].set_ylabel(None)
axs[1].set_ylabel(None)

fig.legends[0].remove()
fig.tight_layout()
fig.savefig(img_path_paper.joinpath("img_signals_detected.pdf"), facecolor=(1, 1, 1, 0))

## Combined Error Plot

In [ ]:
selected_pipelines_for_plotting = [
    ("forouzanfar2018", "drost2022", "none"),
    ("vanlien2013-32-ms", "drost2022", "none"),
    ("forouzanfar2018", "arbol2017-third-derivative", "none"),
    ("forouzanfar2018", "lozano2007-linear-regression", "none"),
]

In [ ]:
algo_levels = ["q_peak_algorithm", "b_point_algorithm", "outlier_correction_algorithm"]

In [ ]:
results_empkins_plot = multi_xs(
    data=results_per_sample_empkins, keys=selected_pipelines_for_plotting, level=algo_levels
)
results_empkins_plot.head()

In [ ]:
results_guardian_plot = multi_xs(
    data=results_per_sample_guardian, keys=selected_pipelines_for_plotting, level=algo_levels
)
results_guardian_plot.head()

In [ ]:
fig, ax = plt.subplots(figsize=(3, 3))

boxplot_algorithm_performance(
    results_empkins_plot,
    metric="absolute_error_per_sample_ms",
    showmeans=True,
    showfliers=False,
    palette=cmaps.faculties_light,
    width=0.9,
    title="EmpkinS Dataset",
    fig=fig,
    ax=ax,
)
ax.set_xticklabels([f"Pipe{i}" for i in range(1, 5)])
ax.set_xlabel(None)
ax.set_ylabel(None)
ax.set_title("Absolute Error", fontdict={"fontweight": "bold"})

fig.tight_layout(pad=0.25)

fig.savefig(img_path_paper.joinpath("img_evaluation_error_single.pdf"), facecolor=(1, 1, 1, 0))

In [ ]:
fig, ax = residual_plot_pep(
    results_empkins_plot,
    selected_pipelines_for_plotting[0],
    alpha=0.5,
    show_upper_limit=True,
    annotate_fontsize="small",
    annotate_bbox=True,
    figsize=(3, 3),
)
ax.set_ylim([-100, 150])
ax.set_title("PEP Pipeline", fontdict={"fontweight": "bold"})
fig.tight_layout(pad=0.25)

fig.savefig(img_path_paper.joinpath("img_residual_plot.pdf"), facecolor=(1, 1, 1, 0))

In [ ]:
fig, ax = residual_plot_pep_heart_rate(
    results_empkins_plot,
    selected_pipelines_for_plotting[0],
    alpha=0.5,
    show_upper_limit=True,
    annotate_fontsize="small",
    annotate_bbox=True,
    figsize=(3, 3),
)
ax.set_ylim([-100, 150])
ax.set_title("PEP Pipeline", fontdict={"fontweight": "bold"})
fig.legends[0].remove()

fig.tight_layout(pad=0.25)

fig.savefig(img_path_paper.joinpath("img_residual_plot_heart_rate.pdf"), facecolor=(1, 1, 1, 0))

In [ ]:
fig, axs = plot_b_point_extraction_forouzanfar2018(
    datapoint_empkins, heartbeat_subset=(5, 7), normalize_time=True, algo_params=None, figsize=(4, 3)
)
fig.legends[0].remove()
axs[0].set_ylabel(None)
axs[1].set_ylabel(None)
axs[0].set_title("Algorithm", fontdict={"fontweight": "bold"})

fig.tight_layout(pad=0.25)
fig.savefig(img_path_paper.joinpath("img_pep_algorithm_insights.pdf"), facecolor=(1, 1, 1, 0))